In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [ ]:
excel_data_df = pd.read_excel(r'../data/data.xlsx')

In [ ]:
excel_data_df['1. Bạn tên là '].tolist() # column data
for idx, header in enumerate(excel_data_df.columns.tolist()):
    print(idx, header)

In [ ]:
excel_data_df.fillna("", inplace = True)

In [ ]:
# Tạo các data theo nhóm cùng độ tương đồng để so sánh khoảng cách euclid sau khi vertor hóa
with open('../data/data.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

In [ ]:
import sys; sys.path.insert(0, '..')
from app.loader import loader
from app.ssmc_fcm import SSMC_FCM

In [35]:
chuyenmon_df = excel_data_df[(excel_data_df["11. Bạn mong muốn tham gia vào ban"] == "Nhân sự")]
data = chuyenmon_df.iloc[:, 21].tolist()
for idx, point in enumerate(data):
    print(idx, point)
dataset = loader.feature_engineering(data)
dataset.shape

0 vc
1 a
2 
3 https://docs.google.com/spreadsheets/d/13UPSeAAnzT79Ld4Gq34XxBUJX2_4Icor/edit?usp=sharing&ouid=100736643351775585814&rtpof=true&sd=true
4 Thành viên giúp b trong lop cntt
5 https://docs.google.com/spreadsheets/d/1oaqyjwYteqj-yqt9GS-8cLr315GjYANgFCuHT9Rb8nA/edit?usp=sharing
6 Đã từng cùng một nhóm bạn thành lập câu lạc bộ tổ chức ôn thi vào 10 cũng như dạy thêm cho các học sinh cấp 2, hợp tác mượn cơ sở của một trung tâm Tiếng Anh, hỗ trợ quảng bá trung tâm với nhiều học sinh hơn.
7 Mình chưa từng viết Plan/MasterPlan nhưng sẵn sàng học hỏi thêm và cải thiện ở mảng này ạ. Mình từng làm nhóm trưởng trong các bài tập nhóm khá nhiều và đã từng lên các timeline hoạt động và nhóm hoạt động khá hiệu quả nên cũng có kinh nghiệm trong việc lập kế hoạch. Hi vọng mình có thể tham gia SINNO và học được việc lập plan một cách chuyên nghiệp và hiệu quả hơn.
8 em chưa có ạ
9 các plan mình đã tham gia không lưu dưới dạng bản mềm và mình cũng không phải là chủ các plan đó nên kiếm lại rất

(30, 768)

In [36]:
supervised_set = [
]
ssmc_fcm = SSMC_FCM(
    dataset=dataset, 
    fields_len=[768] * 1,
    fields_weight=[1],
    n_clusters=5, 
    supervised_set=supervised_set, 
)
ssmc_fcm.clustering()
for cluster in ssmc_fcm.pred_labels:
    print()
    print(cluster)
    for id_data in cluster:
        print(id_data, data[id_data])


[]

[0, 4, 6, 7, 9, 10, 21, 27]
0 vc
4 Thành viên giúp b trong lop cntt
6 Đã từng cùng một nhóm bạn thành lập câu lạc bộ tổ chức ôn thi vào 10 cũng như dạy thêm cho các học sinh cấp 2, hợp tác mượn cơ sở của một trung tâm Tiếng Anh, hỗ trợ quảng bá trung tâm với nhiều học sinh hơn.
7 Mình chưa từng viết Plan/MasterPlan nhưng sẵn sàng học hỏi thêm và cải thiện ở mảng này ạ. Mình từng làm nhóm trưởng trong các bài tập nhóm khá nhiều và đã từng lên các timeline hoạt động và nhóm hoạt động khá hiệu quả nên cũng có kinh nghiệm trong việc lập kế hoạch. Hi vọng mình có thể tham gia SINNO và học được việc lập plan một cách chuyên nghiệp và hiệu quả hơn.
9 các plan mình đã tham gia không lưu dưới dạng bản mềm và mình cũng không phải là chủ các plan đó nên kiếm lại rất khó, có thể coi mình như trang giấy trắng cũng được :v
10 Em chưa có tham gia các Plan/MasterPlan về CNTT. Em đúng nghĩa là chưa biết tí nào về CNTT, vì thế em rất muốn vào CLB để học tập và học hỏi các tiền bối đi trước. Em sẽ l